In [1]:
using Iterators
using Pipe

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [3]:
using RecursiveAutoencoders
using UnfoldingRAE
using Base.Collections
using WordEmbeddings

In [4]:
LL,word_indexes, indexed_words =  load_embeddings("word_emb_data/embeddings-test.txt");
LL[:]./=maximum(abs(LL[:]))
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println



(50,14)
Dict{ASCIIString,Int64}
Array{ASCIIString,1}


In [5]:
training_trees = open("training_sents.jsz","r") do fs
    deserialize(fs)
end;
@pz training_trees

training_trees		Array{Any,1}	(3914,)


In [6]:
training_trees = training_trees[1:10]

10-element Array{Any,1}:
 ((("Pierre","Vinken"),(",",((("61","years"),"old"),","))),(("will",("join",(("the","board"),(("as",("a",("nonexecutive","director"))),("Nov.","29"))))),"."))                                                                                                                                                                                                                   
 (("Mr.","Vinken"),(("is",("chairman",("of",(("Elsevier","N.V."),(",",("the",("Dutch",("publishing","group")))))))),"."))                                                                                                                                                                                                                                                        
 ((("Rudolph","Agnew"),(",",(((("55","years"),"old"),("and",(("former","chairman"),("of",("Consolidated",("Gold",("Fields","PLC"))))))),","))),(("was",("named",("*-1",(("a",("nonexecutive","director")),("of",("this",("British",("indust

In [7]:
function test()
    rr = RAE(LL,word_indexes,indexed_words);

    a=fold(rr,("killer", "cows"))
    typeof(a) |> println
    b=unfold(rr,a);
    @assert b[1].parent==b[2].parent


    a=fold(rr,("the",("killer", "cows")))
    b=unfold(rr,a);
    @assert b[1].parent==b[2].parent.parent==b[3].parent.parent
end
test()


FoldData


In [8]:
function test()
    rr = RAE(LL,word_indexes,indexed_words);
    a=fold(rr,("the",("bad",("killer", "cows"))))
    b=unfold(rr,a);
    @pz rr.W_d
    δd,ΔW_d, Δb_d = UBPTS(rr, b)
    println("-"^54)
    @pz rr.W_d
    @pz ΔW_d
    @pz rr.b_d
    @pz Δb_d

    ΔW_e,Δb_e = UBPTS(rr, a, δd)
    println("+"^54)
    @pz rr.W_e
    @pz ΔW_e
    @pz rr.b_e
    @pz Δb_e

end
test()

rr.W_d		Array{Float64,2}	(100,50)
------------------------------------------------------
rr.W_d		Array{Float64,2}	(100,50)
ΔW_d		Array{Float64,2}	(100,50)
rr.b_d		Array{Float64,1}	(100,)
Δb_d		Array{Float64,1}	(100,)
++++++++++++++++++++++++++++++++++++++++++++++++++++++
rr.W_e		Array{Float64,2}	(50,100)
ΔW_e		Array{Float64,2}	(50,100)
rr.b_e		Array{Float64,1}	(50,)
Δb_e		Array{Float64,1}	(50,)


In [9]:
5÷2

2

In [10]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    
    
    rae.W_e = reshape(θ[1: W_e_len],W_e_shape)
    rae.b_e = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d = reshape(θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len], W_d_shape)
    rae.b_d = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    pack(rae.W_e,rae.b_e, rae.W_d,rae.b_d)
end

@everywhere function pack(∇W_e::NumericMatrix, ∇b_e::NumericVector, ∇W_d::NumericMatrix, ∇b_d::NumericVector)
    [∇W_e[:]; ∇b_e; ∇W_d[:]; ∇b_d] 
end

In [12]:
# tests

function analytic_grad(rae::RAE, tree::Tuple{Any,Any})
    (Δs, err)=loss_and_loss_grad(rae, tree)
    tuple(Δs...)
end
 
function numeric_grad(rae::RAE, tree::Tuple{Any,Any}, ϵ=10.0^-4)
    rae_inner = deepcopy(rae)
    θ = pack(rae_inner)
        Δθ = zeros(size(θ))
    for ii in 1:length(θ)
        ϵᵢ = zeros(size(θ))
        ϵᵢ[ii]=ϵ
        θⁱ⁺ = θ + ϵᵢ
        θⁱ⁻ = θ - ϵᵢ
        
        unpack!(rae_inner,θⁱ⁺)
        Jⁱ⁺ = loss(rae_inner, tree)
        
        unpack!(rae_inner,θⁱ⁻)
        Jⁱ⁻=loss(rae_inner, tree)
        Δθ[ii] = (Jⁱ⁺-Jⁱ⁻)/(2.0*ϵ)
    end
    unpack!(rae_inner,Δθ)
    
    (rae_inner.W_e, rae_inner.b_e, rae_inner.W_d, rae_inner.b_d)
   
end

numeric_grad (generic function with 2 methods)

In [13]:
#eg_tree = training_trees[2]
rae_outer = RAE(LL,word_indexes,indexed_words);
#aW_e, ab_e, aW_d, ab_d = analytic_grad(rae_outer, eg_tree);
#nW_e, nb_e, nW_d, nb_d = numeric_grad(rae_outer, eg_tree, 10.0^-7);


In [14]:
ngs = pack(nW_e, nb_e, nW_d, nb_d)
ags = pack(aW_e, ab_e, aW_d, ab_d)

@printval mean(abs(ngs-ags))
@printval median(abs(ngs-ags))
@printval maximum(abs(ngs-ags))
@printval norm(ngs-ags)
println("-"^54)


for (ng,ag) in zip(ngs, ags)
    println(ng,"\t", ag, "\t", abs(ng-ag))
end




LoadError: LoadError: UndefVarError: nW_e not defined
while loading In[14], in expression starting on line 1

In [16]:
using ForwardDiff

function f(θ)
    rae_inner = RAE(rae_outer.L, rae_outer.word_index, rae_outer.indexed_words,
        similar(rae_outer.W_e),similar(rae_outer.b_e),similar(rae_outer.W_d),similar(rae_outer.b_d))
    unpack!(rae_inner,θ)
    loss(rae_inner, training_trees[2])
end

# Using forwarddiff_jacobian
g = forwarddiff_gradient(f, Float64, fadtype=:dual, n=length(pack(rae_outer)))


g (generic function with 1 method)

In [17]:
θ = pack(rae_outer)
dg = g(θ)

10150-element Array{Float64,1}:
  6.55767e-7
  5.50292e-7
  1.08137e-6
 -1.23558e-6
  1.14172e-6
  4.32851e-7
 -8.97054e-7
  2.34043e-6
  2.30132e-6
 -6.75781e-7
 -1.42362e-6
 -5.32244e-7
 -3.16935e-6
  ⋮         
 -0.00265191
  0.308826  
  0.0307033 
  0.13591   
  0.206062  
  0.0645229 
  0.184008  
  0.144755  
  0.0798861 
 -0.207859  
 -0.208208  
 -0.216134  

In [18]:
ag = pack(analytic_grad(rae_outer, training_trees[2])...)

10150-element Array{Float64,1}:
  6.55767e-7
  5.50292e-7
  1.08137e-6
 -1.23558e-6
  1.14172e-6
  4.32851e-7
 -8.97054e-7
  2.34043e-6
  2.30132e-6
 -6.75781e-7
 -1.42362e-6
 -5.32244e-7
 -3.16935e-6
  ⋮         
 -0.00265191
  0.308826  
  0.0307033 
  0.13591   
  0.206062  
  0.0645229 
  0.184008  
  0.144755  
  0.0798861 
 -0.207859  
 -0.208208  
 -0.216134  

In [19]:
maximum(abs(dg.-ag))


5.551115123125783e-16

In [ ]:
map(t->Vector{t}, [Number, Float16])

In [ ]:
Union(subtypes(Number)...)

In [ ]:
Dual